In [1]:
import xlwings as xw
import pandas as pd

In [2]:
wb1_path = r"C:\Users\raphael.almeida\Documents\Processos\boletos_reemitidos\boletos_reemitidos.xlsx"
wb1 = xw.Book(wb1_path)

ws1 = wb1.sheets['boletos_reemitidos_pagos']
                 
intervalo = ws1.range('A1').expand()

dados = intervalo.value




In [3]:
wb2_path = r"C:\Users\raphael.almeida\Documents\Processos\boletos_reemitidos\Boletos Reemitidos Pagos.xlsx"
wb2 = xw.Book(wb2_path)
ws2 = wb2.sheets['BASE']

ws2.clear_contents()

ws2.range('A1').value = dados

ws3 = wb2.sheets['Boletos Reemitidos Pagos']


In [4]:

#atualizando datas

today = pd.Timestamp.today().normalize()
day_offset_1 = today - pd.Timedelta(days=1)
day_offset_2 = today - pd.Timedelta(days=2)
day_offset_3 = today - pd.Timedelta(days=3)
day_offset_4 = today - pd.Timedelta(days=4)

today



Timestamp('2025-07-01 00:00:00')

In [5]:

ws3.range('E2').value = day_offset_4.date()
ws3.range('H2').value = day_offset_3.date()
ws3.range('K2').value = day_offset_2.date()
ws3.range('N2').value = day_offset_1.date()

In [6]:

#atualizando células de pagamento
base_values = ws2.range('A1').expand().value


In [7]:

# Ajuste: usar a primeira linha como cabeçalho
if base_values:
    df = pd.DataFrame(base_values[1:], columns=base_values[0])
else:
    df = pd.DataFrame()


criando manualmente: listas

In [8]:
listas_ponteiros_4 = df.loc[df['data_reemissao'] == day_offset_4, 'ponteiro'].tolist()
listas_ponteiros_3 = df.loc[df['data_reemissao'] == day_offset_3, 'ponteiro'].tolist()
listas_ponteiros_2 = df.loc[df['data_reemissao'] == day_offset_2, 'ponteiro'].tolist()
listas_ponteiros_1 = df.loc[df['data_reemissao'] == day_offset_1, 'ponteiro'].tolist()

criando automaticamente: celulas pagamento

In [9]:

def calcular_pagamentos_4dias_3empresas(df, listas_ponteiros, empresas):
    """
    Calcula o total de pagamentos para cada empresa ao longo das 4 listas de ponteiros.

    Args:
        df (pd.DataFrame): DataFrame com os dados.
        listas_ponteiros (list): Lista de listas de ponteiros (ex: [listas_ponteiros_1, ..., listas_ponteiros_4]).
        empresas (list): Lista com os nomes das empresas.

    Returns:
        dict: {(empresa, idx_lista): soma_pagamentos}
    """
    resultados = {}
    for idx, ponteiros in enumerate(listas_ponteiros, 1):
        for empresa in empresas:
            soma = df.loc[
                (df['ponteiro'].isin(ponteiros)) &
                (df['empresa'] == empresa),
                'valor_baixa'
            ].sum()
            resultados[(empresa, idx)] = soma
    return resultados

# Exemplo de uso:
listas_ponteiros = [listas_ponteiros_1, listas_ponteiros_2, listas_ponteiros_3, listas_ponteiros_4]
empresas = ['Segtruck', 'Stcoop', 'Viavante']
pagamentos = calcular_pagamentos_4dias_3empresas(df, listas_ponteiros, empresas)
pagamentos


{('Segtruck', 1): np.float64(0.0),
 ('Stcoop', 1): np.float64(0.0),
 ('Viavante', 1): np.float64(0.0),
 ('Segtruck', 2): np.float64(0.0),
 ('Stcoop', 2): np.float64(0.0),
 ('Viavante', 2): np.float64(0.0),
 ('Segtruck', 3): np.float64(0.0),
 ('Stcoop', 3): np.float64(0.0),
 ('Viavante', 3): np.float64(0.0),
 ('Segtruck', 4): np.float64(4288.92),
 ('Stcoop', 4): np.float64(11193.61),
 ('Viavante', 4): np.float64(130436.24)}

teste automático

In [10]:

# Mapeamento de colunas e linhas para cada empresa e dia, seguindo a ordem das variáveis acima
colunas = ['F', 'I', 'L', 'O']  # F: offset 4, H: offset 3, J: offset 2, L: offset 1
linhas = {'Segtruck': 4, 'Stcoop': 5, 'Viavante': 6}
empresas = ['Segtruck', 'Stcoop', 'Viavante']  # ordem das empresas conforme célula acima

# Preencher os valores na ordem correta: para cada coluna (dia), para cada empresa (linha)
# Agora, F6/F7/F8 recebe lista 4, H6/H7/H8 recebe lista 3, J6/J7/J8 recebe lista 2, L6/L7/L8 recebe lista 1
for idx_col, coluna in enumerate(colunas):
    idx_lista = 4 - idx_col  # Inverte a ordem: colunas[0] -> lista 4, colunas[1] -> lista 3, etc.
    for empresa in empresas:
        valor = pagamentos.get((empresa, idx_lista), 0)
        linha = linhas[empresa]
        ws3.range(f'{coluna}{linha}').value = valor


